# Hyperparameter Tuning using HyperDrive

- Install kaggle datasets for easy dataset load
- Load all needed packages
- Prepare workspace and compute cluster

In [1]:
!pip install opendatasets

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core import Dataset
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
import shutil
import opendatasets as od
import pandas as pd
from azureml.widgets import RunDetails

In [2]:
ws = Workspace.from_config()
experiment_name = 'udacity_capstone_hyperdrive'

experiment = Experiment(ws, experiment_name)

In [3]:
cluster_name = "IntensePurposeCluster"

try:
    compute_cluster = ComputeTarget(ws, cluster_name)
    print('existing cluster found')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    
    compute_cluster.wait_for_completion(show_output=True)

existing cluster found


## Dataset

We are using a patient data dataset for covid-19 mortaility from kaggle. We want to try and predict if a patient will die given their circumstances.

- Check if data is already available and download from kaggle if not
- Read and transform data
- Register dataframe as dataset in the workspace

In [5]:
#Check if data exists, else download
if not os.path.exists('covid19-dataset/Covid Data.csv'):
    # Download kaggle covid dataset
    print('downlaoding data')
    od.download('https://www.kaggle.com/datasets/meirnizri/covid19-dataset/', '.')

In [4]:
df = pd.read_csv(r'./covid19-dataset/Covid Data.csv')

In [5]:
#Transform data to get correct format
df['y'] = (df['DATE_DIED'] != '9999-99-99').astype(int)
df['SEX'] = df['SEX'] - 1
df['hospitalized'] = df['PATIENT_TYPE'] - 1
df['tested_positive'] = (df['CLASIFFICATION_FINAL']<4).astype(int)
df = df.drop(columns=['DATE_DIED', 'PATIENT_TYPE', 'CLASIFFICATION_FINAL'])

cols = ['INTUBED', 'PNEUMONIA', 'PREGNANT', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR',
       'HIPERTENSION', 'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY',
       'RENAL_CHRONIC', 'TOBACCO', 'ICU']

for col in cols:
    df[col] = (df[col] == 1).astype(int)

In [6]:
df.head(9)

,USMER,MEDICAL_UNIT,SEX,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,COPD,ASTHMA,...,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,ICU,y,hospitalized,tested_positive
0,2,1,0,0,1,65,0,0,0,0,...,1,0,0,0,0,0,0,1,0,1
1,2,1,1,0,1,72,0,0,0,0,...,1,0,0,1,1,0,0,1,0,0
2,2,1,1,1,0,55,0,1,0,0,...,0,0,0,0,0,0,0,1,1,1
3,2,1,0,0,0,53,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,2,1,1,0,0,68,0,1,0,0,...,1,0,0,0,0,0,0,1,0,1
5,2,1,0,0,1,40,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
6,2,1,0,0,0,64,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,2,1,0,0,1,64,0,1,0,0,...,1,0,0,0,1,0,0,0,0,1
8,2,1,0,0,0,37,0,1,0,0,...,1,0,0,1,0,0,0,0,1,1


In [7]:
datastore = ws.get_default_datastore()

dataset = Dataset.Tabular.register_pandas_dataframe(df,
                target=datastore, 
                name='capstone_dataset', 
                description='Input dataset (Kaggle covid data) for capstone project')

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/a511f0ca-1226-4718-8349-183bfc3c8d89/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## Hyperdrive Configuration

Here we define the parameter optimisation parameters.
- Use a simple Bandit policy to stop optimisation early in case of stalling improvements
- Define search grid for parameters
- Create experiment folder to store run specifc files in and copy in training script
- Define environment for training and configure optimisation run utilising training script

In [11]:
# Utilising a simple Bandit Policy for early termination in case of stagnant performance
early_termination_policy = BanditPolicy(evaluation_interval=3, slack_factor=0.1)

# Defining a search space for the optimal parameters (GradientBoostingTree)
param_sampling = RandomParameterSampling({'learning_rate':uniform(0.01, 1.0),
                             'n_estimators': choice(50, 100, 150, 200, 250, 300, 500),
                             "subsample": choice(0.5, 0.9, 1.0)})

# Create directories to ensure correct model tracking
if experiment_name not in os.listdir():
    os.mkdir("./" + experiment_name + '/')
    shutil.copy2('./covdata_train.py', './'+ experiment_name + '/')

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
# Using script run instead of estimator for the hyperdrive run
src = ScriptRunConfig(source_directory='./'+ experiment_name + '/', 
                      script='covdata_train.py', 
                      compute_target=compute_cluster,
                     arguments=['--learning_rate',0.1, '--n_estimators', 100, '--subsample', '1.0'],
                     environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=param_sampling,
                                    policy=early_termination_policy,
                                    primary_metric_name='AUC_weighted',
                                    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=50,
                                    max_concurrent_runs=4)

In [12]:
# Submiting the experiment
hdr = experiment.submit(config=hyperdrive_run_config)

## Run Details

We test only one model here utilising a SKlearn gradient boosting tree. See details of the run below. 
This cell will only complete once the training run completes such that the entire notebook can be run at once without causing errors.

In [ ]:
RunDetails(hdr).show()
hdr.wait_for_completion()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

Performing interactive authentication. Please follow the instructions on the terminal.
Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


## Best Model

The best model according to the primary metric (F1 score) is selected and registered in the workspace.
We complete the run to make sure there is no bad behaviour.

In [14]:
# Get the best model according to metric and register as model
best_run = hdr.get_best_run_by_primary_metric()
best_run.register_model('covid_death_pred_hdr', 'outputs/Hyperdrive_capstone.joblib')

#Complete run
hdr.complete()

## Model Deployment

Here the model is loaded and prepared for deployment. The imports are done here to separate the deployment part from the training part.
- Load model and workspace
- Define environment and easy entry script for deployment
- Define the deployment config and deploy
- Wait for successful deployment before continuing

In [50]:
from azureml.core.webservice import LocalWebservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, 'covid_death_pred_hdr')

sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
inference_config = InferenceConfig(entry_script="covdata_entry.py", environment=sklearn_env)

# Deploy locally first
deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(workspace=ws, 
                       name='covidsurvival', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config = deployment_config)

local_service.wait_for_deployment(show_output=True)
print(f"Scoring URI is : {local_service.scoring_uri}")

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoint-sdk-v2 /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 3fc95e45fb7841ebb16dd39c91442edf.azurecr.io
Logging into Docker registry 3fc95e45fb7841ebb16dd39c91442edf.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 3fc95e45fb7841ebb16dd39c91442edf.azurecr.io/azureml/azureml_4fada07952d2b43d666b6b63633ffdd5
 ---> 1e1b829da9ea
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 979c19e6e1f1
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjFiOTQ0YTliLWZkYWUtNGY5Ny1hZWIxLWI3ZWVhMGJlYWM1MyIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTIyMzQ2NCIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTIyMzQ2NCIsIndvcmtzcGFjZUlkIjoiM2ZjOTVlNDUtZmI3OC00MWViLWIxNmQtZDM5YzkxNDQyZWRmIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in e7c5d9f0b11f
 ---> f8af836ad45a
Step 4/5 : RUN mv '/var/azureml-app/tmp_wofhmjy.py' /var/azureml-app/main.py


Send a test towards the deployed service to ensure the endpoint working correctly.

In [53]:
import json

data = {"data": [
          {
            "USMER": 2,
            "MEDICAL_UNIT": 1, 
            "SEX": 1,
            "INTUBED": 0,
            "PNEUMONIA": 0,
            "AGE": 35,
            "PREGNANT": 0,
            "DIABETES": 1,
            "COPD": 0,
            "ASTHMA": 0,
            "INMSUPR": 0, 
            "HIPERTENSION": 0,
            "OTHER_DISEASE": 0,
            "CARDIOVASCULAR": 0,
            "OBESITY": 1,
            "RENAL_CHRONIC": 0,
            "TOBACCO": 0,
            "ICU": 0, 
            "hospitalized": 0,
            "tested_positive": 0
          },
          {
            "USMER": 2,
            "MEDICAL_UNIT": 1, 
            "SEX": 1,
            "INTUBED": 1,
            "PNEUMONIA": 1,
            "AGE": 75,
            "PREGNANT": 0,
            "DIABETES": 1,
            "COPD": 0,
            "ASTHMA": 1,
            "INMSUPR": 0, 
            "HIPERTENSION": 0,
            "OTHER_DISEASE": 0,
            "CARDIOVASCULAR": 1,
            "OBESITY": 1,
            "RENAL_CHRONIC": 0,
            "TOBACCO": 0,
            "ICU": 0, 
            "hospitalized": 1,
            "tested_positive": 1
          }]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

In [54]:
import requests

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = 'http://localhost:6789/score'
# If the service is authenticated, set the key or token
#key = ''

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())


[0, 1]


Get the logs of the deployed service and run the last cell to delete it.

In [55]:
local_service.get_logs()

'2023-01-25T10:25:52,396013765+00:00 - rsyslog/run \n2023-01-25T10:25:52,398331968+00:00 - nginx/run \n2023-01-25T10:25:52,400834372+00:00 - iot-server/run \n2023-01-25T10:25:52,405466179+00:00 - gunicorn/run \n2023-01-25T10:25:52,407673382+00:00 | gunicorn/run | \n2023-01-25T10:25:52,410324586+00:00 | gunicorn/run | ###############################################\n2023-01-25T10:25:52,412264389+00:00 | gunicorn/run | AzureML Container Runtime Information\n2023-01-25T10:25:52,417287396+00:00 | gunicorn/run | ###############################################\n2023-01-25T10:25:52,424956607+00:00 | gunicorn/run | \n2023-01-25T10:25:52,427799612+00:00 | gunicorn/run | \n2023-01-25T10:25:52,432181618+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20221010.v9\n2023-01-25T10:25:52,433880220+00:00 | gunicorn/run | \n2023-01-25T10:25:52,435799423+00:00 | gunicorn/run | \n2023-01-25T10:25:52,437695726+00:00 | gunicorn/run | PATH environment variable

In [ ]:
local_service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

